In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import random
from ew import EWBase, EWDense, EWConv2D

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
def enable_ew(model):
    for layer in model.layers:
        if isinstance(layer, EWBase):
            layer.enable()

def disable_ew(model):
    for layer in model.layers:
        if isinstance(layer, EWBase):
            layer.disable()

In [3]:
def to_float(x, y):
    return tf.cast(x, tf.float32) / 255.0, y

dataset = tfds.load("mnist", as_supervised=True, split="train")
val_set = tfds.load("mnist", as_supervised=True, split="test")

dataset = dataset.map(to_float)
val_set = val_set.map(to_float)

Generate the key set. In the paper they took a subset of the dataset and assigned random labels to them in order to combat query modification. However that altered the validation accuracy too much. For simplicity reasons we will just invert the pixels of a subset of the training dataset and assign a random label to it.

In [4]:
def invert(x, y):
    return tf.abs(x - 1.0), tf.convert_to_tensor(random.randint(0, 9), dtype=tf.int64)

key_set = dataset.take(128)
key_set = key_set.map(invert)
dataset = dataset.skip(128)

An easy way to achieve a high accuracy on the key set is to overfit our model on the key set, since it doesn't have to generalize.

In [5]:
key_set = key_set.concatenate(key_set).concatenate(key_set).concatenate(key_set).concatenate(key_set).concatenate(key_set)

union = dataset.concatenate(key_set)

dataset = dataset.shuffle(2048).batch(128).prefetch(AUTOTUNE)
union = union.shuffle(2048).batch(128).prefetch(AUTOTUNE)
val_set = val_set.batch(128)

t is the 'temperature' hyperparameter. The higher t is, the more the values of the weight matrix will get squeezed, 2.0 was used in the paper.

In [6]:
t = 2.0

model = keras.Sequential([
            EWConv2D(16, 3, t, padding="same", activation=keras.activations.relu),
            EWConv2D(32, 3, t, padding="same", strides=2, activation=keras.activations.relu),
            EWConv2D(64, 3, t, padding="same", strides=2, activation=keras.activations.relu),
            keras.layers.Flatten(),
            EWDense(10, activation=None, t=t)
        ])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["sparse_categorical_accuracy"])
model.build(input_shape=(None, 28, 28, 1))

Train the model normally with exponential weighting disabled until it converges:

In [7]:
_ = model.fit(x=dataset, epochs=3, validation_data=val_set)

Epoch 1/3
468/468 [==============================] - 10s 21ms/step - loss: 0.4735 - sparse_categorical_accuracy: 0.8533 - val_loss: 0.1299 - val_sparse_categorical_accuracy: 0.9613
Epoch 2/3
468/468 [==============================] - 9s 20ms/step - loss: 0.1229 - sparse_categorical_accuracy: 0.9641 - val_loss: 0.1012 - val_sparse_categorical_accuracy: 0.9691
Epoch 3/3
468/468 [==============================] - 9s 19ms/step - loss: 0.0913 - sparse_categorical_accuracy: 0.9727 - val_loss: 0.0779 - val_sparse_categorical_accuracy: 0.9753


Enable exponential weighting and train the model on the union of the dataset and the key set in order to embed the watermark:

In [8]:
enable_ew(model)
_ = model.fit(x=union, epochs=2, validation_data=val_set)

Epoch 1/2
474/474 [==============================] - 9s 19ms/step - loss: 0.0753 - sparse_categorical_accuracy: 0.9776 - val_loss: 0.1095 - val_sparse_categorical_accuracy: 0.9646
Epoch 2/2
474/474 [==============================] - 9s 19ms/step - loss: 0.0672 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.0645 - val_sparse_categorical_accuracy: 0.9794


Reset the optimizer. Disable exponential weighting and test the accuracy on the key set:

In [9]:
model.optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
disable_ew(model)
_, key_acc = model.evaluate(key_set.batch(128))
_, val_acc = model.evaluate(val_set)

print(f"Watermark accuracy is {round(key_acc * 100, 2)}%.")
print(f"Validation set accuracy is {round(val_acc * 100, 2)}%.")

79/79 [==============================] - 1s 7ms/step - loss: 0.0645 - sparse_categorical_accuracy: 0.9794
Watermark accuracy is 100.0%.
Validation set accuracy is 97.94%.


If the watermark accuracy(key_acc) is above a predefined threshold, the model was watermarked by us.